# 漏斗图

In [1]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Funnel, Page

In [2]:
dt = pd.read_excel('loudou.xlsx')

In [3]:
dt

,环节,人数,占位数据,每环节转化率,总体转化率
0,总进件数,533,1.0,1.0000,100.0
1,准入规则,531,131.0,0.9962,99.6
2,黑名单,271,197.0,0.5104,50.8
3,反欺诈,139,229.5,0.5129,26.1
4,信用模型,74,266.5,0.5324,13.9


In [8]:
funnel = Funnel()
funnel.add('环节',[list(z) for z in zip(dt['环节'],dt['总体转化率'])],label_opts=opts.LabelOpts(formatter="{c}%"))
funnel.set_global_opts(title_opts=opts.TitleOpts(title="总体转化漏斗图",pos_left='center'),
                       legend_opts=opts.LegendOpts(pos_right='left',pos_top='bottom',orient='vertical'),
                       toolbox_opts=opts.ToolboxOpts(is_show=True,pos_left=0,pos_top='bottom',orient='vertical'))
funnel.render_notebook()


# RFM

## 导入数据

In [9]:
import pandas as pd
from datetime import datetime

In [10]:
rfm_dt = pd.read_excel('PYTHON-RFM实战数据.xlsx')

In [11]:
rfm_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28833 entries, 0 to 28832
Data columns (total 9 columns):
品牌名称    28833 non-null object
买家昵称    28833 non-null object
付款日期    28833 non-null datetime64[ns]
订单状态    28833 non-null object
实付金额    28833 non-null int64
邮费      28833 non-null int64
省份      28833 non-null object
城市      28832 non-null object
购买数量    28833 non-null int64
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 2.0+ MB


## 查看订单状态

In [12]:
rfm_dt.groupby(by='订单状态')['买家昵称'].count()

订单状态
交易成功                 27793
付款以后用户退款成功，交易自动关闭     1040
Name: 买家昵称, dtype: int64

- 订单状态分布为：
    - 交易成功                        ：27793
    - 付款以后用户退款成功，交易自动关闭  ：1040

## 删除退款数据

In [13]:
new_df = rfm_dt[rfm_dt['订单状态']=='交易成功']

In [14]:
new_df.groupby(by='订单状态')['买家昵称'].count()

订单状态
交易成功    27793
Name: 买家昵称, dtype: int64

## 提取：买家昵称，付款时间和实付金额

In [15]:
new_df2 = new_df[['买家昵称','付款日期','实付金额']]
new_df2.head()

,买家昵称,付款日期,实付金额
0,叫我李2,2019-01-01 00:17:59,186
1,0cyb1992,2019-01-01 00:59:54,145
2,萝污萌莉,2019-01-01 07:48:48,194
5,重碎叠,2019-01-01 10:00:07,197
6,iho_jann,2019-01-01 10:00:08,168


## 拿到所有用户最近一次付款时间

In [16]:
latest_order_by_buyer = new_df.groupby(by='买家昵称')['付款日期'].max().reset_index()
latest_order_by_buyer.head()

,买家昵称,付款日期
0,.blue_ram,2019-02-04 17:49:34.000
1,.christiny,2019-01-29 14:17:15.000
2,.willn1,2019-01-11 03:46:18.000
3,.托托m,2019-01-11 02:26:33.000
4,0000妮,2019-06-28 16:53:26.458


## 计算R值

In [17]:
today = datetime.now().strftime('%Y-%m-%d')

In [18]:
latest_order_by_buyer['R']= (pd.to_datetime(today)-latest_order_by_buyer['付款日期']).dt.days

In [19]:
rfm = latest_order_by_buyer[['买家昵称','R']]

In [20]:
rfm.head()

,买家昵称,R
0,.blue_ram,484
1,.christiny,490
2,.willn1,508
3,.托托m,508
4,0000妮,340


## 统计购买次数

In [21]:
new_df['日期标签'] = new_df['付款日期'].astype(str).str[:10]

/Users/mason_tao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
new_df.head()

,品牌名称,买家昵称,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量,日期标签
0,数据不吹牛,叫我李2,2019-01-01 00:17:59,交易成功,186,6,上海,上海市,1,2019-01-01
1,数据不吹牛,0cyb1992,2019-01-01 00:59:54,交易成功,145,0,广东省,广州市,1,2019-01-01
2,数据不吹牛,萝污萌莉,2019-01-01 07:48:48,交易成功,194,8,山东省,东营市,1,2019-01-01
5,数据不吹牛,重碎叠,2019-01-01 10:00:07,交易成功,197,0,江苏省,南京市,1,2019-01-01
6,数据不吹牛,iho_jann,2019-01-01 10:00:08,交易成功,168,0,广东省,广州市,1,2019-01-01


In [23]:
buy_time_by_date = new_df.groupby(by=['买家昵称','日期标签'])['付款日期'].count().reset_index()

In [24]:
buy_time_by_date.head()

,买家昵称,日期标签,付款日期
0,.blue_ram,2019-02-04,1
1,.christiny,2019-01-29,1
2,.willn1,2019-01-11,1
3,.托托m,2019-01-11,1
4,0000妮,2019-06-28,1


In [25]:
buy_time_by_date.columns = ['买家昵称','日期标签','F']

In [26]:
buy_time_by_date.head()

,买家昵称,日期标签,F
0,.blue_ram,2019-02-04,1
1,.christiny,2019-01-29,1
2,.willn1,2019-01-11,1
3,.托托m,2019-01-11,1
4,0000妮,2019-06-28,1


## 拿到用户平均支付金额，并输出每个买家的RFM

In [27]:
total_money = new_df.groupby(by='买家昵称')['实付金额'].sum().reset_index()

In [28]:
total_money.head()

,买家昵称,实付金额
0,.blue_ram,49
1,.christiny,183
2,.willn1,34
3,.托托m,37
4,0000妮,164


In [29]:
rfm = rfm.merge(right=total_money,how='inner')

In [30]:
rfm = rfm.merge(right=buy_time_by_date,how='inner')

In [31]:
rfm.head()

,买家昵称,R,实付金额,日期标签,F
0,.blue_ram,484,49,2019-02-04,1
1,.christiny,490,183,2019-01-29,1
2,.willn1,508,34,2019-01-11,1
3,.托托m,508,37,2019-01-11,1
4,0000妮,340,164,2019-06-28,1


In [32]:
rfm['M'] = rfm['实付金额']/rfm['F']

In [33]:
rfm = rfm[['买家昵称','R','F','M']]

In [34]:
rfm.head()

,买家昵称,R,F,M
0,.blue_ram,484,1,49.0
1,.christiny,490,1,183.0
2,.willn1,508,1,34.0
3,.托托m,508,1,37.0
4,0000妮,340,1,164.0


$\color{blue}{R值与题目中结果不一样应该是定义的今日日期不一样} $